In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [35]:
allTransactions = np.array([np.array([int(id) for id in line.split()]) for line in open("kosarak.dat")])

In [36]:
occurences = {}

for i in range(len(allTransactions)):
    for j in range(len(allTransactions[i])):
        if not allTransactions[i][j] in occurences:
            occurences[allTransactions[i][j]] = 0
        occurences[allTransactions[i][j]] += 1

In [37]:
max(occurences.keys()) # because we need highiest 1000 values is better to sort than to find them one by one

41270

In [42]:
sortedTransactions = sorted(occurences,key=occurences.__getitem__, reverse=True)

In [48]:
T = 1000
translations = {}
retranslations = {}
transData = np.array(sortedTransactions[:T])
for i in range(T):
    translations[i] = transData[i]
    retranslations[transData[i]] = i

In [51]:
data = np.zeros((T,T))
for line in allTransactions:
    importantTrans = [retranslations[i] for i in line if i in transData]
    for transId in importantTrans:
        for transId2 in importantTrans:
            if transId2 != transId:
                data[transId,transId2] += 1

In [52]:
def averageDistToCenter(X,C):
    avdist = np.sum(np.sqrt(np.sum((X - C)**2,axis=0)),axis=0)
    avdist /= np.maximum(np.size(X,axis=1),1)
    return avdist

def KMeans(K, X, doPrinting=False):
    d = np.size(X,axis=0)
    N = np.size(X,axis=1)
    I = np.eye(K)
    R = X[:,np.random.randint(0,N,K)] #random start centers
    M = np.zeros((d,K)).astype(int)
    changes = True
    iters = 0
    while changes:
#         D = np.sum((R.T[np.newaxis,:,:] - X.T[:,np.newaxis,:])**2., axis=-1).T # distances from centers
        D = -2. * np.dot(X.T,R) + np.sum(R**2,axis=0,keepdims=True)# it is not necessary to add X^2
        m = np.argmin(D, axis=1)
        newM = np.take(I,m,axis=0).astype(int) # matrix of membership
        changes = not np.array_equal(M,newM)
        M = newM
        R = np.dot(X,M)/np.maximum(np.sum(M.T,axis=1),1) # new centers
        iters +=1
    if doPrinting:
        avDistToCenter = np.array([averageDistToCenter(X[:,M[:,i] == 1], R[:,i,np.newaxis]) for i in range(K)])
        groupSizes = np.sum(M.T,axis=1)
        print('Average distance to center within groups:\n',avDistToCenter)
        print("Iterations: ", iters, "\n")
        print("Group sizes: ", groupSizes, "\n")
    return R, M, m, groupSizes

In [53]:
data = data.T

In [124]:
K = 10
R, M, m, sizes = KMeans(K, data, doPrinting=True)

Average distance to center within groups:
 [   5591.18567547   54344.9277638   221715.084201     11202.27631296
    4297.27399458    6534.73448426    2638.57089197    2201.26699028
   14219.01361517    1821.66812251]
Iterations:  36 

Group sizes:  [ 35   5   3  23 108  51 350 401   7  17] 



In [125]:
for i in range(T):
    if m[i] == 8:
        print(translations[i])

4
55
64
77
2
138
83


### Wcześniej na przekątnej macierzy odległości mieliśmy zera, teraz będziemy tam trzymać liczbę występowań danego produktu w transakcjach

In [84]:
data1 = np.zeros((T,T))
for line in allTransactions:
    importantTrans = [retranslations[i] for i in line if i in transData]
    for transId in importantTrans:
        for transId2 in importantTrans:
            data1[transId,transId2] += 1

In [85]:
data1 = data1.T

In [153]:
K = 25
R, M, m, sizes = KMeans(K, data1, doPrinting=True)

Average distance to center within groups:
 [   6081.62488489    3149.68846909    2166.26302556    2386.89386753
   12607.04535321    2737.74363055    1966.75419504   45273.11051555
    6469.9147516    19537.02256516    5095.73250697    2402.85993299
       0.            3297.30079866    2492.08978891    8963.04077253
  211834.28539285    2075.5518236    89036.9394518    17499.42306611
    3998.40888502    3837.97502475       0.            3555.65223895
    2646.75536389]
Iterations:  23 

Group sizes:  [ 52  28  54  57  14  78  92   7  33  14  19 105   0  34  13  30   3 177
   5  12  41  62   1  53  16] 



In [155]:
for i in range(T):
    if m[i] == 16:
        print(translations[i])

6
3
11


### Za każdym razem pojawiają się grupy wielkości 3 i 5, czasem grupa wielkości 7. Drugi sposób wydaje się je wskazywać w sposób stabilniejszy

In [156]:
T = 500
translations = {}
retranslations = {}
transData = np.array(sortedTransactions[:T])
for i in range(T):
    translations[i] = transData[i]
    retranslations[transData[i]] = i

In [157]:
data500 = np.zeros((T,T))
for line in allTransactions:
    importantTrans = [retranslations[i] for i in line if i in transData]
    for transId in importantTrans:
        for transId2 in importantTrans:
            data500[transId,transId2] += 1

In [201]:
K = 10
R, M, m, sizes = KMeans(K, data500, doPrinting=True)

Average distance to center within groups:
 [   2474.32880326   88787.36930363    4610.76653344   33448.78469266
  211705.11722792    4499.46921716   14001.82636153    3729.00950563
    8421.22909458   14289.78579368]
Iterations:  16 

Group sizes:  [ 15   5 189  18   3 173  22   6  58  11] 



In [175]:
for i in range(T):
    if m[i] == 7:
        print(translations[i])

4
55
64
77
2
138
83


In [182]:
K = 15
R, M, m, sizes = KMeans(K, data500, doPrinting=True)

Average distance to center within groups:
 [   6299.89016133   29899.34447606   59021.44897008    3902.32331099
    2451.98782567   18133.18304864    2474.32880326   89254.80078599
  211705.11722792    6011.97304527    9290.21658671    3277.21894817
    3934.29406236   17446.96231162    7707.77634481]
Iterations:  25 

Group sizes:  [ 22  16   3  79  13   8  15   3   3  55  29 133  79  11  31] 



In [178]:
for i in range(T):
    if m[i] == 2:
        print(translations[i])

1
218
7
27
148


In [183]:
data500_1 = np.zeros((T,T))
for line in allTransactions:
    importantTrans = [retranslations[i] for i in line if i in transData]
    for transId in importantTrans:
        for transId2 in importantTrans:
            if transId2 != transId:
                data500_1[transId,transId2] += 1

In [194]:
K = 10
R, M, m, sizes = KMeans(K, data500_1, doPrinting=True)

Average distance to center within groups:
 [   3038.15458684    3136.81850762    3657.03148174   53955.60929707
   14015.43832425   10549.46768317  221565.19365283    7382.22773104
    2597.94724716    5377.29700913]
Iterations:  19 

Group sizes:  [192  33  82   5   7  22   3  25  97  34] 



In [188]:
for i in range(T):
    if m[i] == 2:
        print(translations[i])

6
3
11


### Grupa wielkości 7 zawsze występuje w drugim sposobie, zatem będziemy teraz z niego korzystać

In [202]:
T = 100
translations = {}
retranslations = {}
transData = np.array(sortedTransactions[:T])
for i in range(T):
    translations[i] = transData[i]
    retranslations[transData[i]] = i

In [230]:
data100 = np.zeros((T,T))
for line in allTransactions:
    importantTrans = [retranslations[i] for i in line if i in transData]
    for transId in importantTrans:
        for transId2 in importantTrans:
            data100[transId,transId2] += 1

In [237]:
K = 15
R, M, m, sizes = KMeans(K, data100, doPrinting=True)

Average distance to center within groups:
 [   7939.99071745   11669.96299596   87986.09144258   21048.88380865
   15683.90864178    8575.30116701       0.           12580.25602879
  211160.63320013       0.           44993.28277953    4745.54957302
       0.               0.            7939.74842516]
Iterations:  8 

Group sizes:  [ 3  9  5  9 10 20  1  2  3  1  7  2  1  0 27] 



### Grupa wielkości 7 znowu mało stabilna

In [238]:
T = 200
translations = {}
retranslations = {}
transData = np.array(sortedTransactions[:T])
for i in range(T):
    translations[i] = transData[i]
    retranslations[transData[i]] = i

In [240]:
data200 = np.zeros((T,T))
for line in allTransactions:
    importantTrans = [retranslations[i] for i in line if i in transData]
    for transId in importantTrans:
        for transId2 in importantTrans:
            data200[transId,transId2] += 1

In [265]:
K = 10
R, M, m, sizes = KMeans(K, data200, doPrinting=True)

Average distance to center within groups:
 [   6192.58543832   45083.13950544   14237.56238577   88363.60487011
    7223.39648102    9423.3455126    20058.83448754    6599.98644675
       0.          211433.44144097]
Iterations:  9 

Group sizes:  [74  7 10  5 42 34 18  6  1  3] 



In [255]:
K = 5
R, M, m, sizes = KMeans(K, data200, doPrinting=True)

Average distance to center within groups:
 [   9086.63826326   88363.60487011   30497.43123797  211433.44144097
     551.57388071]
Iterations:  14 

Group sizes:  [163   5  26   3   3] 



### Przy dzieleniu na zbyt mało kawałków znowu zdarza nam się gubić grupę 7